In [2]:
# import required packages 
import matplotlib.dates as mpdates 
import matplotlib.pyplot as plt 
import mplfinance as mpf
import matplotlib as mpl
from PIL import Image
import pandas as pd 
import math as math
import numpy as np
import io   as io
import gc   as gc
import os as os
from numpy import savetxt

#set run instance number, and the total number of concurrent instances
run=1
tot_runs = 1; 

#timeframe
tf = 32

#set_pixels
img_size=32

#set directory
direct =  "D:/1 - School/Econometrics/2020 - 2021/Scriptie/Explainable AI/Scripts/Data/binary_colour_scheme.32frames" #"C:/Users/robin/1 - Scriptie/images/intraday/binary_colour_scheme.64frames"

#loading the data
data = pd.read_csv(r'D:\1 - School\Econometrics\2020 - 2021\Scriptie\Explainable AI\Scripts\Data\stock_data_clean.csv',header=[0, 1] , index_col = 0 )
data.index=pd.to_datetime(data.index)
data = np.log(data) - np.log(data.shift(1))

#loading the data
data_volume = pd.read_csv(r'D:\1 - School\Econometrics\2020 - 2021\Scriptie\Explainable AI\Scripts\Data\stock_data_clean.csv',header=[0, 1] , index_col = 0 )
data_volume.index=pd.to_datetime(data_volume.index)
data_volume = np.log(data_volume) - np.log(data_volume.shift(1))

#subsetting the data
total_symbols = math.floor(len(data.columns.unique(level=0))/tot_runs)
symbols = data.columns.unique(level=0)[(run-1)*total_symbols:run*total_symbols]




In [3]:
def make_binary_array(i,j,data,data_volume,symbols,direct,img_size, tf):
    #slice the data in to #tf trading day windows
    data_temp=data[symbols[j]][i-tf:i]

    #find price data and 
    price_data = data_temp.Close
    volume_data = data_volume[symbols[j]][i-tf:i].Volume  

    #min-max the data
    price_max = np.amax(price_data)
    price_min = np.amin(price_data)
    price_data_min_max = np.divide ((price_data - price_min),(price_max-price_min))

    vol_max = np.amax(volume_data)
    vol_min = np.amin(volume_data)
    vol_data_min_max = np.divide ((volume_data - vol_min),(vol_max-vol_min))

    #bin the data
    num_bins = int((tf-2)/2)

    price_bins = np.linspace(np.amin(price_data_min_max),np.amax(price_data_min_max),num=num_bins)
    price_bin_index = np.digitize(price_data_min_max,price_bins)-1

    vol_bins = np.linspace(np.amin(vol_data_min_max),np.amax(vol_data_min_max),num=num_bins)
    vol_bin_index = np.digitize(vol_data_min_max,vol_bins)-1

    #create the binary array
    output = np.zeros((tf,tf))

    for i in range(0,tf):
            output[num_bins-(price_bin_index[i]+1),i]+=1
            output[tf-vol_bin_index[i]-1,i]+=1
    return output

In [4]:
#check if folder excists, if not, create it. 
if not os.path.exists(direct):
    os.mkdir(direct)

for j in range(0,len(symbols)):

    for i in range(tf,len(data)):

        #check if the file has already been created
        if not os.path.exists(direct+"/"+str(symbols[j])+"_"   +str(i-tf+1)+".txt"):
            #call the functions and create the 
             binary_array = make_binary_array(i , j , data, data_volume, symbols ,direct , img_size,tf)
             pd.DataFrame(binary_array).to_csv(direct+"/"+str(symbols[j])+"_"   +str(i-tf+1)+".txt", sep=' ', index=False, header=False)